In [63]:
import pandas as pd
import json

df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services_blue_removed.xlsx')

/tmp/ipykernel_198697/4288358939.py:4: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('UNSPSC_English.csv')


In [64]:
df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:] #setter

In [65]:
# import nltk and download resources
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/diz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/diz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [66]:
# parse the stop words and special characters out
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stop_words.add(':')
stop_words.add('--')
stop_words.add('-')
stop_words.add('(')
stop_words.add(')')
stop_words.add('/')
stop_words.add('*')
stop_words.add(';')
stop_words.add('.')
stop_words.add('i.e')
stop_words.add('A')
stop_words.add('a')
stop_words.add(',')
stop_words.add("'s")

In [67]:
# word parsing function
def createParsedColumns(file, col1, col2, start_index, new_col):
    for index1, description1 in enumerate(file[col1], start_index):
        if not pd.isna(description1):
            word_tokens1 = word_tokenize(description1)

            filtered_sentence1 = []
            for w1 in word_tokens1:
                if w1 not in stop_words:
                    # make all the words into lowercase, it is easier for comparison
                    filtered_sentence1.append(w1.lower())

            # remove empty strings
            while "" in filtered_sentence1:
                filtered_sentence1.remove("")

            # updating record in col1
            file.at[index1, col1]= ','.join(filtered_sentence1)
        else:
            file.at[index1, col1]= ''

    for index2, description2 in enumerate(file[col2], start_index):
        if not pd.isna(description2):
            word_tokens2 = word_tokenize(description2)

            filtered_sentence2 = []
            for w2 in word_tokens2:
                if w2 not in stop_words:
                    filtered_sentence2.append(w2.lower())

            # remove empty strings
            while "" in filtered_sentence2:
                filtered_sentence2.remove("")

            # updating record in col2
            file.at[index2, col2]= ','.join(filtered_sentence2)
        else:
            file.at[index2, col2]= ''

    # create 3rd column
    new_column_series = []
    for index3, description3 in enumerate(file[col2], start_index):
        temp1 = file[col1].loc[index3].split(',')

        if not pd.isna(description3):
            temp2 = file[col2].loc[index3].split(',')
            temp3 = temp1 + temp2
            temp3 = set(temp3)

            if '' in temp3:
                temp3.remove('')

            new_column_series.append(list(temp3))
        else:
            new_column_series.append(list(temp1))

    # insert new column
    file.loc[:,new_col] = new_column_series

In [68]:
column_name = 'AvaTax System Tax Code Description'
column_name_2 = 'Additional AvaTax System Tax Code Information'
new_col_name = 'tax_parsed'
createParsedColumns(df_xlsx, column_name, column_name_2, 2, new_col_name)
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information,tax_parsed
2,DA010000,"digital,goods,advertising,material,digital,phy...","tangible,personal,property,designs,artwork,pho...","[goods, property, personal, physical, tangible..."
3,DA030000,"digital,goods,art,digital,physical,media","canned,artwork,transferred,customer,electronic...","[goods, transferred, means, canned, art, physi..."
4,DA040000,"digital,goods,audio,digital,physical,media","audio,music,spoken,word,transferred,customer,e...","[goods, transferred, means, music, spoken, phy..."
5,DA040100,"radio,subscriptions","subscriptions,satellite,radio,services","[subscriptions, satellite, services, radio]"
6,DA051011,"digital,audio,works,sold,end,user,rights,perma...",,"[rights, user, permanent, sold, audio, works, ..."
...,...,...,...,...
2381,SY010100,"yoga,classes,yoga,studio,yoga,classes,offered",,"[studio, yoga, offered, classes]"
2382,SY010200,"yoga,classes,facilities,offer,services,yoga,cl...",,"[yoga, offer, classes, facilities, services]"
2383,SL090200,"lodging,accommodations",,"[lodging, accommodations]"
2384,ST112506,"travel,agency,fees",,"[travel, fees, agency]"


In [69]:
# tc_hash_map = {}
#
# new_col_name = 'tax_parsed'
#
# for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):
#
#     # if tax_code = NaN
#     if isinstance(tax_code, float):
#         prefix = df_xlsx["AvaTax System Tax Code"].loc[index + 1]
#         level_1 = prefix[0:2]
#         level_2 = prefix[2:4]
#         level_3 = "0000"
#         df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
#     else:
#         level_1 = tax_code[0:2]
#         level_2 = tax_code[2:4]
#         level_3 = tax_code[4:8]
#
#
#     if level_1 not in tc_hash_map:
#         temp_dict0 = {level_3: df_xlsx[new_col_name].loc[index]}
#         temp_dict = {level_2: [df_xlsx[new_col_name].loc[index], temp_dict0]}
#         tc_hash_map[level_1] = [df_xlsx[new_col_name].loc[index], temp_dict]
#     elif level_2 not in tc_hash_map[level_1][1]:
#         temp_dict0 = {level_3: df_xlsx[new_col_name].loc[index]}
#         tc_hash_map[level_1][1][level_2] = [df_xlsx[new_col_name].loc[index], temp_dict0]
#     elif level_3 not in tc_hash_map[level_1][1][level_2][1]:
#         tc_hash_map[level_1][1][level_2][1][level_3] = df_xlsx[new_col_name].loc[index]
#
# print(json.dumps(tc_hash_map, sort_keys=True, indent=2))


In [70]:
# write all the words from avalara file into a table

tc_table = {}
new_col_name = 'tax_parsed'
for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):
    if not isinstance(tax_code, float):
        tc_table[tax_code] = df_xlsx[new_col_name].loc[index]
print(json.dumps(tc_table, sort_keys=False, indent=2))

{
  "DA010000": [
    "goods",
    "property",
    "personal",
    "physical",
    "tangible",
    "electronic",
    "material",
    "media",
    "etc",
    "customer",
    "photos",
    "services",
    "associated",
    "artwork",
    "transferred",
    "means",
    "advertising",
    "designs",
    "digital"
  ],
  "DA030000": [
    "goods",
    "transferred",
    "means",
    "canned",
    "art",
    "physical",
    "media",
    "digital",
    "customer",
    "electronic",
    "artwork"
  ],
  "DA040000": [
    "goods",
    "transferred",
    "means",
    "music",
    "spoken",
    "physical",
    "media",
    "audio",
    "digital",
    "customer",
    "word",
    "electronic"
  ],
  "DA040100": [
    "subscriptions",
    "satellite",
    "services",
    "radio"
  ],
  "DA051011": [
    "rights",
    "user",
    "permanent",
    "sold",
    "audio",
    "works",
    "digital",
    "use",
    "end"
  ],
  "DA051012": [
    "rights",
    "conditioned",
    "continued",
    "audio",
 

In [71]:
# Convert row to header and remove the row
df_csv = df_csv.rename(columns=df_csv.iloc[8])
df_csv = df_csv.loc[9:] # setter
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Column1', 'Synonym', 'Acronym'])
df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition
9,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,NaN,NaN,NaN
12,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101501,Cats,NaN
13,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101502,Dogs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
156402,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,NaN,NaN,NaN
156403,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,95141901.0,Medical unit,A building or structure where medical equipmen...
156404,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,95141902.0,Laboratory unit,A building or structure where laboratory equip...
156405,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,95141903.0,Dental unit,A building or structure where dental equipment...


In [ ]:
column_name = 'Segment Title'
column_name_2 = 'Segment Definition'
column_name_3 = 'Family Title'
column_name_4 = 'Family Definition'
column_name_5 = 'Class Title'
column_name_6 = 'Class Definition'
column_name_7 = 'Commodity Title'
column_name_8 = 'Definition'
createParsedColumns(df_csv, column_name, column_name_2, 9, 'segment_parsed')
createParsedColumns(df_csv, column_name_3, column_name_4, 9, 'family_parsed')
createParsedColumns(df_csv, column_name_5, column_name_6, 9, 'class_parsed')
createParsedColumns(df_csv, column_name_7, column_name_8, 9, 'commodity_parsed')
df_csv

In [ ]:
unspsc_tc_table_lvl_1 = {}
unspsc_tc_table_lvl_2 = {}
unspsc_tc_table_lvl_3 = {}
unspsc_tc_table_lvl_4 = {}

seg_new_name = 'segment_parsed'
fam_new_name = 'family_parsed'
class_new_name = 'class_parsed'
com_new_name = 'commodity_parsed'

for index, com_code in enumerate(df_csv['Commodity'], 9):

    if isinstance(com_code, float):
        continue
    else:
        level_1 = com_code[0:2] 
        level_2 = com_code[0:4]
        level_3 = com_code[0:6]
        level_4 = com_code[0:8]

    if level_1 not in unspsc_tc_table_lvl_1:
        unspsc_tc_table_lvl_1[level_1] = df_csv[seg_new_name].loc[index]

    if level_2 not in unspsc_tc_table_lvl_2:
        unspsc_tc_table_lvl_2[level_2] = df_csv[fam_new_name].loc[index]

    if level_3 not in unspsc_tc_table_lvl_3:
        unspsc_tc_table_lvl_3[level_3] = df_csv[class_new_name].loc[index]     

    if level_4 not in unspsc_tc_table_lvl_4:
        unspsc_tc_table_lvl_4[level_4] = df_csv[com_new_name].loc[index]

print(json.dumps(unspsc_tc_table_lvl_1, sort_keys=True, indent=2))
print(json.dumps(unspsc_tc_table_lvl_2, sort_keys=True, indent=2))
print(json.dumps(unspsc_tc_table_lvl_3, sort_keys=True, indent=2))
print(json.dumps(unspsc_tc_table_lvl_4, sort_keys=True, indent=2))

In [ ]:
import sys

def my_print(text):
    sys.stdout.write(str(text))
    sys.stdout.flush()

In [ ]:

seg_word_matches_dict = {}

# checking if words in avalara file matches with words in segment title

# use for loop to loop through each row
for avalara_code, word_array in tc_table.items():
    # check if array is empty
    if word_array:
        # set max words matches to the one row of the avalara file to 0
        seg_max_match = 0
        # keep track of the key with the max number of matches
        seg_max_key = 0
        # use for loop to look through each key and segment title pair
        for seg_key, seg_array in unspsc_tc_table_lvl_1.items():
            # check if array is empty
            if seg_array:
                # every loop of comparing words in one of the segment title reset the match counter.
                seg_match_counter = 0
                # use for loop to loop through each word in one array
                for word in word_array:
                    for seg in seg_array:
                        if word == seg:
                            seg_match_counter = seg_match_counter + 1
                if seg_match_counter > seg_max_match:
                    seg_max_match = seg_match_counter
                    seg_max_key = seg_key

        seg_word_matches_dict[avalara_code] = seg_max_key

filtered_seg_word_matches_array = {}
for key, value in seg_word_matches_dict.items():
    if value != 0:
        filtered_seg_word_matches_array[key] = value

print(json.dumps(filtered_seg_word_matches_array, sort_keys=False, indent=2))


In [ ]:
# because the seg keys are just a few therefor we could set the seg key as keys in the new dict
unique_seg_keys = set(filtered_seg_word_matches_array.values())

seg_key_with_avalara_code_value = {}
for key in unique_seg_keys:
    seg_key_with_avalara_code_value[key] = [code for code in filtered_seg_word_matches_array.keys() if filtered_seg_word_matches_array[code] == key]

print(json.dumps(seg_key_with_avalara_code_value, sort_keys=False, indent=2))

In [ ]:

fam_word_matches_dict = {}
for seg_level_1_key, avalara_code_value_array in seg_key_with_avalara_code_value.items():

    for avalara_code in avalara_code_value_array:
        word_array = tc_table[avalara_code]

        fam_max_match = 0
        fam_max_key = 0

        for fam_level_2_key, fam_level_2_value_array in unspsc_tc_table_lvl_2.items():
            # check if the level 2 array is empty and only go through the family titles that are children of the segments
            if  fam_level_2_value_array and fam_level_2_key[0:2] == seg_level_1_key:

                if word_array:
                    fam_match_count = 0

                    for word in word_array:
                        for fam in fam_level_2_value_array:
                            if word == fam:
                                fam_match_count = fam_match_count + 1

                    if fam_match_count > fam_max_match:
                        fam_max_match = fam_match_count
                        fam_max_key = fam_level_2_key


        fam_word_matches_dict[avalara_code] = fam_max_key

filtered_fam_word_matches_array = {}
for key, value in fam_word_matches_dict.items():
    if value != 0:
        filtered_fam_word_matches_array[key] = value


print(json.dumps(filtered_fam_word_matches_array, sort_keys=False, indent=2))




In [ ]:
unique_fam_keys = set(filtered_fam_word_matches_array.values())
fam_key_with_avalara_code_value = {}
for key in unique_fam_keys:
    fam_key_with_avalara_code_value[key] = [code for code in filtered_fam_word_matches_array.keys() if filtered_fam_word_matches_array[code] == key]

print(json.dumps(fam_key_with_avalara_code_value, sort_keys=False, indent=2))

In [ ]:
class_word_matches_dict = {}
for fam_level_2_key, avalara_code_value_array in fam_key_with_avalara_code_value.items():

    for avalara_code in avalara_code_value_array:
        word_array = tc_table[avalara_code]

        class_max_match = 0
        class_max_key = 0

        for class_level_3_key, class_level_3_value_array in unspsc_tc_table_lvl_3.items():
            # check if the level 2 array is empty and only go through the family titles that are children of the segments
            if  class_level_3_value_array and class_level_3_key[0:4] == fam_level_2_key:

                if word_array:
                    class_match_count = 0

                    for word in word_array:
                        for cla in class_level_3_value_array:
                            if word == cla:
                                class_match_count = class_match_count + 1

                    if class_match_count > class_max_match:
                        class_max_match = class_match_count
                        class_max_key = class_level_3_key


        class_word_matches_dict[avalara_code] = class_max_key

filtered_class_word_matches_array = {}
for key, value in class_word_matches_dict.items():
    if value != 0:
        filtered_class_word_matches_array[key] = value


print(json.dumps(filtered_class_word_matches_array, sort_keys=False, indent=2))

In [ ]:
unique_class_keys = set(filtered_class_word_matches_array.values())
class_key_with_avalara_code_value = {}
for key in unique_class_keys:
    class_key_with_avalara_code_value[key] = [code for code in filtered_class_word_matches_array.keys() if filtered_class_word_matches_array[code] == key]

print(json.dumps(class_key_with_avalara_code_value, sort_keys=False, indent=2))

In [ ]:

comm_word_matches_dict = {}
for class_level_3_key, avalara_code_value_array in class_key_with_avalara_code_value.items():

    for avalara_code in avalara_code_value_array:
        word_array = tc_table[avalara_code]

        comm_max_match = 0
        comm_max_key = 0

        for comm_level_4_key, comm_level_4_value_array in unspsc_tc_table_lvl_4.items():
            # check if the level 2 array is empty and only go through the family titles that are children of the segments
            if  comm_level_4_value_array and comm_level_4_key[0:6] == class_level_3_key:

                if word_array:
                    comm_match_count = 0

                    for word in word_array:
                        for comm in comm_level_4_value_array:
                            if word == comm:
                                comm_match_count = comm_match_count + 1

                    if comm_match_count > comm_max_match:
                        comm_max_match = comm_match_count
                        comm_max_key = comm_level_4_key


        comm_word_matches_dict[avalara_code] = comm_max_key

filtered_comm_word_matches_array = {}
for key, value in comm_word_matches_dict.items():
    if value != 0:
        filtered_comm_word_matches_array[key] = value


print(json.dumps(filtered_comm_word_matches_array, sort_keys=False, indent=2))



In [ ]:
avalara_code = 'AvaTax System Tax Code'
avalara_name = 'AvaTax System Tax Code Description'
comm_title = 'Commodity Title'
comm = 'Commodity'
result_data = []
for key, value in comm_word_matches_dict.items():
    result_data.append([value, df_csv.loc[df_csv[comm] == value, comm_title], key, df_xlsx.loc[df_xlsx[avalara_code] == key, avalara_name]])

result_df = pd.DataFrame(result_data, columns=[comm, comm_title, avalara_code, avalara_name])
result_drop = result_df[result_df['Commodity'] == 0].index
result_df.drop(result_drop, inplace=True)
result_df.drop_duplicates('Commodity', inplace=True)
result_df

In [ ]:
# unique_comm_keys = set(filtered_comm_word_matches_array.values())
# comm_key_with_avalara_code_value = {}
# for key in unique_comm_keys:
#     comm_key_with_avalara_code_value[key] = [code for code in filtered_comm_word_matches_array.keys() if filtered_comm_word_matches_array[code] == key]
#
# print(json.dumps(comm_key_with_avalara_code_value, sort_keys=False, indent=2))

In [ ]:
# from collections import defaultdict
#
# factory = lambda: defaultdict(factory)
#
# unspc_segments = defaultdict(factory)
# unspc_family = defaultdict(factory)
# unspc_class = defaultdict(factory)
# unspc_commodity = defaultdict(factory)
#
# unspc_family_full = defaultdict(factory)
# unspc_class_full = defaultdict(factory)
# unspc_commodity_full = defaultdict(factory)
#
# for index in range(9, len(df_csv.index)):
#     if not pd.isna(df_csv['Commodity'].iloc[index]):
#         seg_key = str(df_csv['Commodity'].iloc[index])[:2]
#         fam_key = str(df_csv['Commodity'].iloc[index])[2:4]
#         class_key = str(df_csv['Commodity'].iloc[index])[4:6]
#         com_key = str(df_csv['Commodity'].iloc[index])[6:8]
#     elif not pd.isna(df_csv['Class'].iloc[index]):
#         seg_key = str(df_csv['Class'].iloc[index])[:2]
#         fam_key = str(df_csv['Class'].iloc[index])[2:4]
#         class_key = str(df_csv['Class'].iloc[index])[4:6]
#         com_key = ""
#     elif not pd.isna(df_csv['Family'].iloc[index]):
#         seg_key = str(df_csv['Family'].iloc[index])[:2]
#         fam_key = str(df_csv['Family'].iloc[index])[2:4]
#         class_key = ""
#         com_key = ""
#     elif not pd.isna(df_csv['Segment'].iloc[index]):
#         seg_key = str(df_csv['Segment'].iloc[index])[:2]
#         fam_key = ""
#         class_key = ""
#         com_key = ""
#     else:
#         seg_key = ""
#         fam_key = ""
#         class_key = ""
#         com_key = ""
#
#     # segments hash map
#     if seg_key:
#         if seg_key not in unspc_segments:
#             unspc_segments[seg_key] = df_csv['segment_parsed'].iloc[index]
#
#     # family hash map
#     if seg_key and fam_key:
#         if seg_key not in unspc_family:
#             unspc_family[seg_key][fam_key] = df_csv['family_parsed'].iloc[index]
#         elif fam_key not in unspc_family[seg_key]:
#             unspc_family[seg_key][fam_key] = df_csv['family_parsed'].iloc[index]
#
#         if seg_key + fam_key not in unspc_family_full:
#             unspc_family_full[seg_key + fam_key] = df_csv['family_parsed'].iloc[index]
#
#     # class hash map
#     if seg_key and fam_key and class_key:
#         if seg_key not in unspc_class:
#             unspc_class[seg_key][fam_key][class_key] = df_csv['class_parsed'].iloc[index]
#         elif fam_key not in unspc_class[seg_key]:
#             unspc_class[seg_key][fam_key][class_key] = df_csv['class_parsed'].iloc[index]
#         elif class_key not in unspc_class[seg_key][fam_key]:
#             unspc_class[seg_key][fam_key][class_key] = df_csv['class_parsed'].iloc[index]
#
#         if seg_key + fam_key + class_key not in unspc_class_full:
#             unspc_class_full[seg_key + fam_key + class_key] = df_csv['class_parsed'].iloc[index]
#
#     # commodity hash map
#     if seg_key and fam_key and class_key and com_key:
#         if seg_key not in unspc_class:
#             unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
#         elif fam_key not in unspc_commodity[seg_key]:
#             unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
#         elif class_key not in unspc_commodity[seg_key][fam_key]:
#             unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
#         elif com_key not in unspc_commodity[seg_key][fam_key][class_key]:
#             unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
#
#         if seg_key + fam_key + class_key + com_key not in unspc_class_full:
#             unspc_commodity_full[seg_key + fam_key + class_key + com_key] = df_csv['commodity_parsed'].iloc[index]
#
# with open("segments.json", "w") as outfile:
#     json.dump(unspc_segments, outfile)
#
# with open("family.json", "w") as outfile:
#     json.dump(unspc_family, outfile)
#
# with open("class.json", "w") as outfile:
#     json.dump(unspc_class, outfile)
#
# with open("commodity.json", "w") as outfile:
#     json.dump(unspc_commodity, outfile)

# print(json.dumps(unspc_segments, sort_keys=True, indent=2))
# print(json.dumps(unspc_family, sort_keys=True, indent=2))
# print(json.dumps(unspc_class, sort_keys=True, indent=2))
# print(json.dumps(unspc_commodity, sort_keys=True, indent=2))



In [ ]:
# unspc_segments
# unspc_family
# unspc_class
# unspc_commodity
# new_column_series = []
#
# segment_best_matches = {}
# family_best_matches = {}
# class_best_matches = {}
# commodity_best_matches = {}
#
# for segment_key, segment_keywords in unspc_segments.items():
#     for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
#         num_matches = len(np.intersect1d(keyword_arr_avalara, segment_keywords))
#         if num_matches > 1:
#             tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
#             if segment_key in segment_best_matches:
#                 segment_best_matches[segment_key].append(tax_code)
#             else:
#                 segment_best_matches[segment_key] = [tax_code]
#
# for family_key, family_keywords in unspc_family_full.items():
#     for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
#         num_matches = len(np.intersect1d(keyword_arr_avalara, family_keywords))
#         if num_matches > 0:
#             tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
#             if family_key in family_best_matches:
#                 family_best_matches[family_key].append(tax_code)
#             else:
#                 family_best_matches[family_key] = [tax_code]
#
# for class_key, class_keywords in unspc_class_full.items():
#     for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
#         num_matches = len(np.intersect1d(keyword_arr_avalara, class_keywords))
#         if num_matches > 0:
#             tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
#             if class_key in class_best_matches:
#                 class_best_matches[class_key].append(tax_code)
#             else:
#                 class_best_matches[class_key] = [tax_code]
#
# for commodity_key, commodity_keywords in unspc_commodity_full.items():
#     for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
#         num_matches = len(np.intersect1d(keyword_arr_avalara, commodity_keywords))
#         if num_matches > 0:
#             tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
#             if commodity_key in commodity_best_matches:
#                 commodity_best_matches[commodity_key].append(tax_code)
#             else:
#                 commodity_best_matches[commodity_key] = [tax_code]
#
# print(segment_best_matches)
# print(family_best_matches)
# print(class_best_matches)
# print(commodity_best_matches)
#
#



    # df.insert(df.shape[1], new_col, new_column_series)


